In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

Q1

In [24]:
def y(x):
    return 2*x**2+4

In [ ]:
# nn model
class QuadraticNet(nn.Module):
    def __init__(self):
        super(QuadraticNet, self).__init__()
        self.fc1 = nn.Linear(1, 32)    
        self.act1 = nn.ReLU()          
        self.fc2 = nn.Linear(32, 16)   
        self.act2 = nn.ReLU()          
        self.fc3 = nn.Linear(16, 1)    

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)   
        x = self.fc2(x)
        x = self.act2(x)   
        x = self.fc3(x)
        return x

model = QuadraticNet()
print(model)


QuadraticNet(
  (fc1): Linear(in_features=1, out_features=32, bias=True)
  (act1): ReLU()
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (act2): ReLU()
  (fc3): Linear(in_features=16, out_features=1, bias=True)
)


In [26]:
x = torch.linspace(0, 10, steps=101).unsqueeze(1)
y = y(x)

In [27]:
# loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# train
epochs = 2000
for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()

    if epoch % 200 == 0:
        print(f"Epoch {epoch}, Loss = {loss.item():.6f}")

# test: x = 3
test_x = torch.tensor([[3.0]])
pred_y = model(test_x)

# True value
true_y = 2 * 3**2 + 4

print("\nTest result:")
print("Predicted y =", pred_y.item())
print("True y      =", true_y)

Epoch 0, Loss = 8449.348633
Epoch 200, Loss = 2.110577
Epoch 400, Loss = 0.100999
Epoch 600, Loss = 0.036892
Epoch 800, Loss = 0.024610
Epoch 1000, Loss = 0.151506
Epoch 1200, Loss = 0.045918
Epoch 1400, Loss = 0.031488
Epoch 1600, Loss = 0.027908
Epoch 1800, Loss = 0.024824

Test result:
Predicted y = 22.086084365844727
True y      = 22


Q2

In [13]:
data = np.load(r"D:\KAUST\semster2\ML_IN_GEO\SiesmicEventsClassification_Normalized.npz")

In [14]:
X = data['data']  # 特征数据
y = data['label']  # 标签数据

print(X.shape)
print(y.shape)


(1000, 6000)
(1000,)


In [15]:
X_freq = np.fft.fft(X, axis=1) 
X_freq_real = X_freq.real
X_freq_imag = X_freq.imag
X_freq_combined = np.concatenate([X_freq_real, X_freq_imag], axis=1)

In [16]:
from sklearn.model_selection import train_test_split

# 假设 X_freq.shape = (样本数, 特征数)，y.shape = (样本数,)
X_train, X_test, y_train, y_test = train_test_split(
    X_freq_combined, y, test_size=0.2, random_state=42, shuffle=True
)
# 转为 PyTorch 张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor  = torch.tensor(y_test, dtype=torch.long)

In [17]:
class SeismicNet(nn.Module):
    def __init__(self, input_size, hidden_layers=[64, 32], output_size=2):
        super().__init__()
        layers = []
        last_size = input_size
        for h in hidden_layers:
            layers.append(nn.Linear(last_size, h))
            layers.append(nn.ReLU())
            last_size = h
        layers.append(nn.Linear(last_size, output_size))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ==============================
# 5. 训练和测试函数
# ==============================
def train_model(model, X_train, y_train, epochs=20, batch_size=32, lr=0.001):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    model.train()
    
    for epoch in range(epochs):
        permutation = torch.randperm(X_train.size()[0])
        correct = 0
        total = 0
        for i in range(0, X_train.size()[0], batch_size):
            indices = permutation[i:i+batch_size]
            batch_x, batch_y = X_train[indices], y_train[indices]
            
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            # 计算 batch 准确率
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == batch_y).sum().item()
            total += batch_y.size(0)
        
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}/{epochs}, Accuracy: {epoch_acc:.4f}")
        
    return model

def test_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        preds = torch.argmax(outputs, dim=1)
        correct = (preds == y_test).sum().item()
        acc = correct / y_test.size(0)
    return acc

In [53]:
# ==============================
# 6. 网格搜索隐藏层配置
# ==============================
hidden_layer_configs = [
    # 单隐藏层
    [16],
    [32],
    [64],
    [128],
    [256],

    # 两隐藏层
    [32, 16],
    [64, 32],
    [128, 64],
    [256, 128],
    [128, 32],
    [64, 16],

    # 三隐藏层
    [64, 32, 16],
    [128, 64, 32],
    [256, 128, 64],
    [128, 64, 16],
    [64, 32, 16],

    # 四隐藏层（比较深）
    [128, 64, 32, 16],
    [256, 128, 64, 32],
    [64, 64, 32, 16],
]


best_acc = 0
best_config = None

input_size = X_train_tensor.shape[1]

for config in hidden_layer_configs:
    print(f"\nTraining with hidden layers: {config}")
    model = SeismicNet(input_size=input_size, hidden_layers=config)
    model = train_model(model, X_train_tensor, y_train_tensor, epochs=200, batch_size=64, lr=0.001)
    acc = test_model(model, X_test_tensor, y_test_tensor)
    print(f"Test Accuracy: {acc:.4f}")
    
    if acc > best_acc:
        best_acc = acc
        best_config = config

print("\n=============================")
print("Best hidden layer config:", best_config)
print("Best test accuracy:", best_acc)


Training with hidden layers: [16]
Epoch 1/200, Accuracy: 0.4888
Epoch 2/200, Accuracy: 0.8788
Epoch 3/200, Accuracy: 0.9550
Epoch 4/200, Accuracy: 0.9812
Epoch 5/200, Accuracy: 0.9938
Epoch 6/200, Accuracy: 0.9938
Epoch 7/200, Accuracy: 0.9962
Epoch 8/200, Accuracy: 0.9962
Epoch 9/200, Accuracy: 0.9962
Epoch 10/200, Accuracy: 0.9962
Epoch 11/200, Accuracy: 0.9962
Epoch 12/200, Accuracy: 0.9962
Epoch 13/200, Accuracy: 0.9962
Epoch 14/200, Accuracy: 0.9962
Epoch 15/200, Accuracy: 0.9962
Epoch 16/200, Accuracy: 0.9962
Epoch 17/200, Accuracy: 0.9962
Epoch 18/200, Accuracy: 0.9962
Epoch 19/200, Accuracy: 0.9962
Epoch 20/200, Accuracy: 0.9962
Epoch 21/200, Accuracy: 0.9962
Epoch 22/200, Accuracy: 0.9962
Epoch 23/200, Accuracy: 0.9962
Epoch 24/200, Accuracy: 0.9962
Epoch 25/200, Accuracy: 0.9962
Epoch 26/200, Accuracy: 0.9962
Epoch 27/200, Accuracy: 0.9962
Epoch 28/200, Accuracy: 0.9962
Epoch 29/200, Accuracy: 0.9962
Epoch 30/200, Accuracy: 0.9962
Epoch 31/200, Accuracy: 0.9962
Epoch 32/200,

In [18]:
import torch

# ==============================
# 设备配置
# ==============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 将训练和测试张量搬到 GPU（如果有）
X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
X_test_tensor  = X_test_tensor.to(device)
y_test_tensor  = y_test_tensor.to(device)

# ==============================
# 网格搜索超参数
# ==============================
hidden_layer_configs = [
    # 单隐藏层
    [16], [32], [64], [128], [256],

    # 两隐藏层
    [32, 16], [64, 32], [128, 64], [256, 128], [128, 32], [64, 16],

    # 三隐藏层
    [64, 32, 16], [128, 64, 32], [256, 128, 64], [128, 64, 16], [64, 32, 16],

    # 四隐藏层
    [128, 64, 32, 16], [256, 128, 64, 32], [64, 64, 32, 16],
]

batch_sizes = [16, 32, 64]
learning_rates = [0.001, 0.005, 0.01]

best_acc = 0
best_config = None
best_batch = None
best_lr = None

input_size = X_train_tensor.shape[1]

# ==============================
# 训练 & 测试循环
# ==============================
for config in hidden_layer_configs:
    for batch_size in batch_sizes:
        for lr in learning_rates:
            print(f"\nTraining with hidden layers: {config}, batch size: {batch_size}, lr: {lr}")
            
            # 初始化模型并搬到 GPU
            model = SeismicNet(input_size=input_size, hidden_layers=config)
            model = model.to(device)
            
            # 训练
            model = train_model(model, X_train_tensor, y_train_tensor,
                                epochs=20, batch_size=batch_size, lr=lr)
            
            # 测试
            acc = test_model(model, X_test_tensor, y_test_tensor)
            print(f"Test Accuracy: {acc:.4f}")
            
            # 更新最优配置
            if acc > best_acc:
                best_acc = acc
                best_config = config
                best_batch = batch_size
                best_lr = lr

print("\n=============================")
print("Best hidden layer config:", best_config)
print("Best batch size:", best_batch)
print("Best learning rate:", best_lr)
print("Best test accuracy:", best_acc)


Using device: cuda

Training with hidden layers: [16], batch size: 16, lr: 0.001
Epoch 1/20, Accuracy: 0.4950
Epoch 2/20, Accuracy: 0.8525
Epoch 3/20, Accuracy: 0.9463
Epoch 4/20, Accuracy: 0.9725
Epoch 5/20, Accuracy: 0.9862
Epoch 6/20, Accuracy: 0.9950
Epoch 7/20, Accuracy: 0.9962
Epoch 8/20, Accuracy: 0.9962
Epoch 9/20, Accuracy: 0.9962
Epoch 10/20, Accuracy: 0.9962
Epoch 11/20, Accuracy: 0.9962
Epoch 12/20, Accuracy: 0.9962
Epoch 13/20, Accuracy: 0.9962
Epoch 14/20, Accuracy: 0.9962
Epoch 15/20, Accuracy: 0.9962
Epoch 16/20, Accuracy: 0.9962
Epoch 17/20, Accuracy: 0.9962
Epoch 18/20, Accuracy: 0.9962
Epoch 19/20, Accuracy: 0.9962
Epoch 20/20, Accuracy: 0.9962
Test Accuracy: 0.5250

Training with hidden layers: [16], batch size: 16, lr: 0.005
Epoch 1/20, Accuracy: 0.4587
Epoch 2/20, Accuracy: 0.6850
Epoch 3/20, Accuracy: 0.8313
Epoch 4/20, Accuracy: 0.9150
Epoch 5/20, Accuracy: 0.9437
Epoch 6/20, Accuracy: 0.9725
Epoch 7/20, Accuracy: 0.9900
Epoch 8/20, Accuracy: 0.9850
Epoch 9/20, 

In [19]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import torch
# 切换到评估模式
model.eval()

with torch.no_grad():
    outputs = model(X_test_tensor)           # 输出 logits
    preds = torch.argmax(outputs, dim=1)    # 取预测类别
preds_cpu = preds.cpu().numpy()
y_test_cpu = y_test_tensor.cpu().numpy()


In [20]:
# 混淆矩阵
cm = confusion_matrix(y_test_cpu, preds_cpu)
print("Confusion Matrix:")
print(cm)

# 精确率、召回率、F1 分数
precision = precision_score(y_test_cpu, preds_cpu, average='weighted')
recall = recall_score(y_test_cpu, preds_cpu, average='weighted')
f1 = f1_score(y_test_cpu, preds_cpu, average='weighted')

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")


Confusion Matrix:
[[60 42]
 [53 45]]
Precision: 0.5242
Recall:    0.5250
F1 Score:  0.5230
